# Train and eval Datasets Generation

In [ ]:
import json
import os
import csv
import re
import random
from datetime import datetime, timedelta
from tqdm import tqdm

## Four datasets prompts construction

In [ ]:
MODE = "train"
DATA = "PMData" # "PMData", "GLOBEM", "AW_FB"

def avg(_list):
    return sum(_list) / len(_list)

def json_reader(file_name):
    f = open(file_name)
    data = json.load(f)
    f.close()
    return data

def csv_reader(file_name):
    f = open(file_name, 'r')
    reader = csv.reader(f)
    # f.close()
    return reader

def convert_to_lowercase(input_string):
    return input_string.lower()

def has_alphabets(input_string):
    return any(char.isalpha() for char in input_string)

def has_numbers(input_string):
    has_numbers = any(char.isdigit() for char in input_string)
    return has_numbers

def extract_words_inside_brackets(input_string):
    pattern = r"\[\'([^\']+)\'\]|\[\"([^\"]+)\"\]"

    matches = re.findall(pattern, input_string)

    extracted_words = ' '.join([word for match in matches for word in match if word])

    return extracted_words

In [ ]:
import pandas as pd

data_local_path_header = '' # Your data_local_path_header

if MODE == "train":
    if DATA == "PMData":
        print("[INFO] Generating datasets for PMData ...")
        """ 
            1) Readiness Prediction 
            2) Stress Estimation
            3) Calorie Consumption Prediction (food image) 
            4) Sleep Quality Prediction
        """
        participant_info = {
            'p01': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p02': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p03': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p04': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p05': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p06': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p07': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p08': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p09': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p10': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p11': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p12': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p13': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p14': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p15': [-1, -1, 'N/A', 'N/A', 'N/A'],
            'p16': [-1, -1, 'N/A', 'N/A', 'N/A'],
        }
        
        DATA_PATH = data_local_path_header + "data/pmdata"
        final_data_list = []
        
        fatigue_records = []
        readiness_records = []
        stress_records = []
        sleep_quality_records = []
        for SUBTASK in ['sleep_quality', 'stress', 'readiness', 'fatigue']:
            index = 0
            subtask_data = []
            true_labels = []
            print("[INFO] Subtask:", SUBTASK)
            for dir1 in tqdm(os.listdir(DATA_PATH)):
                if "." in dir1:
                    continue
                tmp = participant_info[dir1]
                age = tmp[0]
                height = str(tmp[1]) + " cm"
                gender = tmp[2]
                fpath1 = os.path.join(DATA_PATH, dir1)

                if '.' not in dir1:      
                    # print("[INFO] Participant:", dir1)      
                    for dir2 in os.listdir(fpath1):
                        fpath2 = os.path.join(fpath1, dir2)

                        if dir2 == 'fitbit':
                            # [1] fitbit
                            # 1-1) calories.json
                            # 1-2) distance.json
                            
                            # 1-3) exercise.json
                            exercise_data = json_reader(fpath2 + '/exercise.json')
                            
                            # 1-4) heart_rate.json
                            try:
                                heart_rate_data = json_reader(fpath2 + '/resting_heart_rate.json')
                            except:
                                continue
                            
                            # 1-5) sleep_score.csv
                            # 1-6) sleep.json
                            sleep_data = json_reader(fpath2 + '/sleep.json')

                            # 1-7) steps.json
                            # 1-8) time_in_heart_rate_zones.json
                        
                        elif dir2 == 'pmsys':

                            wellness_data = csv_reader(fpath2 + "/wellness.csv")
                            wellness_dict = {k:[] for k in ['effective_time_frame', 'fatigue', 'mood', 'readiness', 'sleep_duration_h', 'sleep_quality', 'soreness', 'soreness_area', 'stress']}
                            for i,data in enumerate(wellness_data):
                                # ['effective_time_frame', 'fatigue', 'mood', 'readiness', 'sleep_duration_h', 'sleep_quality', 'soreness', 'soreness_area', 'stress']
                                if i == 0:
                                    continue

                                date = data[0][:10] + "_" + data[0][11:][:-1].split(".")[0]
                                fatigue = data[1]
                                mood = data[2]
                                readiness = data[3]
                                sleep_dur = data[4]
                                sleep_qual = data[5]
                                # soreness = data[6]
                                # soreness_area = data[7]
                                stress = data[-1]

                                wellness_dict['effective_time_frame'].append(date)
                                wellness_dict['fatigue'].append(fatigue)
                                wellness_dict['mood'].append(mood)
                                wellness_dict['readiness'].append(readiness)
                                wellness_dict['sleep_duration_h'].append(sleep_dur)
                                wellness_dict['sleep_quality'].append(sleep_qual)
                                wellness_dict['stress'].append(stress)

                    
                    date = wellness_dict['effective_time_frame']
                    fatigue = wellness_dict['fatigue']
                    mood = wellness_dict['mood']
                    readiness = wellness_dict['readiness']
                    sleep_dur = wellness_dict['sleep_duration_h']
                    sleep_qual = wellness_dict['sleep_quality']
                    stress = wellness_dict['stress']

                    
                    for d,f,m,r,sd,sq,s in zip(date, fatigue, mood, readiness, sleep_dur, sleep_qual, stress):
                        # print("[INFO] Readiness:", r)
                        new_d = datetime.strptime(d, '%Y-%m-%d_%H:%M:%S')
                        # print("[INFO] target date:", new_d)

                        # [1] mood 
                        mood = m
                        stress = s
                        sleep_quality = sq
                        
                        # [2] exercise
                        exercise_hist = []
                        while True:
                            # TODO: datetime compare / 7-days or 14-days? -> refer to fitbit premium / heart rate
                            for e_data in exercise_data:
                                # print(e_data)
                                # print()
                                e_date = e_data['startTime'][:10] + "_" + e_data['startTime'][11:]
                                new_ed = datetime.strptime(e_date, '%Y-%m-%d_%H:%M:%S')
                                # print("[INFO] e_data's date:", new_ed)

                                # compare the date and make 2-weeks window
                                if (new_d > new_ed) and (new_d - new_ed) < timedelta(days=14):

                                    try:
                                        activity = e_data['activityName']
                                        burn_calories = float(e_data['calories'])
                                        steps = float(e_data['steps'])
                                        duration = float(e_data['duration']) / 1000 / 60 # in minutes
                                        exercise_hist.append([new_ed, activity, duration, burn_calories, steps])
                                    except:
                                        continue
                                    
                            break                 
                        
                        # [3] sleep
                        sleep_hist = []
                        while True:
                            for s_data in sleep_data:
                                # print(s_data)
                                # print()
                                s_date = s_data['startTime'][:10] + "_" + s_data['startTime'][11:]
                                new_sd = datetime.strptime(s_date, '%Y-%m-%d_%H:%M:%S')

                                # compare the date and make 2-weeks window
                                if (new_d > new_sd) and (new_d - new_sd) < timedelta(days=14):
                                    # print("[INFO2]", new_sd, "belongs")

                                    sleep_duration = float(s_data['duration']) / 1000 / 60 # in minutes
                                    sleep_hist.append([new_sd, sleep_duration])

                            break

                        # [4] heart rate
                        hr_hist = []
                        while True:
                            for hr_data in heart_rate_data:
                                # print(hr_data)
                                hr_date = hr_data['dateTime'][:10] + "_" + hr_data['dateTime'][11:]
                                new_hrd = datetime.strptime(hr_date, '%Y-%m-%d_%H:%M:%S')
                                # print("[INFO] e_data's date:", new_ed)
    
                                # compare the date and make 2-weeks window
                                if (new_d > new_hrd) and (new_d - new_hrd) < timedelta(days=14):
                                    # print("[INFO3]", new_hrd, "belongs")
                                    rhr = float(hr_data['value']['value'])
                                    hr_hist.append([new_hrd, rhr])
                            break

                        # encode historical data
                        try:
                            steps_14d = sum([x[-1] for x in exercise_hist]) / len(exercise_hist)
                        except:
                            steps_14d = "N/A"
                            continue
                        try:
                            calories_14d = sum([x[-2] for x in exercise_hist]) / len(exercise_hist)
                        except:
                            calories_14d = "N/A"
                            continue
                        try:
                            rhr_14d = sum([x[-1] for x in hr_hist]) / len(hr_hist)
                        except:
                            rhr_14d = "N/A"
                            continue
                        try:
                            sleep_dur_14d = sum([x[-1] for x in sleep_hist]) / len(sleep_hist)
                        except:
                            sleep_dur_14d = "N/A"
                            continue

                        if SUBTASK == "readiness":
                            range1 = 0
                            range2 = 10
                        elif SUBTASK == "stress":
                            range1 = 1
                            range2 = 5
                        elif SUBTASK == "sleep_quality":
                            range1 = 1
                            range2 = 5 
                        elif SUBTASK == "fatigue":
                            range1 = 1
                            range2 = 5

                        # few-shot chain of thoughts context enhancement
                        # I = "You are a personalized healthcare agent trained to predict {} which ranges from {} to {} based on physiological data and user information.".format(SUBTASK, range1, range2)        
                        Q = "The recent 14-days sensor readings show: [Steps]: {} steps, [Burned Calorories]: {} calories, [Resting Heart Rate]: {} beats/min, [SleepMinutes]: {} minutes, [Mood]: {} out of 5; What would be the predicted {}?".format([x[-1] for x in exercise_hist], [x[-2] for x in exercise_hist], [x[-1] for x in hr_hist], [x[-1] for x in sleep_hist], m, SUBTASK)
                        # I = "You are a personalized healthcare agent trained to predict {} which ranges from 1 to 5 based on physiological data and user information."
                
                        # NEW
                        if SUBTASK == "readiness":
                            # zero-shot context giving - instruction style
                            # I = "Given the user's context and sensor data, predict the readiness score (ranges from 0 to 10)"
                            I = "You are a personalized healthcare agent trained to predict readiness score which ranges from 0 to 10 based on physiological data and user information."
                            A = "The predicted {} level is {}.".format(SUBTASK, r)
                            # readiness_records.append({'index':index,'Steps':[x[-1] for x in exercise_hist], 'Burned Calorories':[x[-2] for x in exercise_hist], 'Resting Heart Rate(beats/min)':[x[-1] for x in hr_hist], 'SleepMinutes':[x[-1] for x in sleep_hist], 'Mood':m, 'Readiness':r})

                        elif SUBTASK == "stress":
                            # I = "Given the user's context and sensor data, predict the stress level (ranges from 0 to 5)"
                            I = "You are a personalized healthcare agent trained to predict stress level which ranges from 0 to 5 based on physiological data and user information."
                            A = "The predicted {} level is {}.".format(SUBTASK, s)
                            # stress_records.append({'index':index,'Steps':[x[-1] for x in exercise_hist], 'Burned Calorories':[x[-2] for x in exercise_hist], 'Resting Heart Rate(beats/min)':[x[-1] for x in hr_hist], 'SleepMinutes':[x[-1] for x in sleep_hist], 'Mood':m, 'Stress':s})

                        elif SUBTASK == "sleep_quality":
                            # I = "Given the user's context and sensor data, predict the sleep quality (ranges from 1 to 5)"
                            I = "You are a personalized healthcare agent trained to predict sleep quality which ranges from 1 to 5 based on physiological data and user information."
                            A = "The predicted {} level is {}.".format(SUBTASK, sq)
                            # sleep_quality_records.append({'index':index,'Steps':[x[-1] for x in exercise_hist], 'Burned Calorories':[x[-2] for x in exercise_hist], 'Resting Heart Rate(beats/min)':[x[-1] for x in hr_hist], 'SleepMinutes':[x[-1] for x in sleep_hist], 'Mood':m, 'Sleep_quality':sq})
                            
                        elif SUBTASK == "fatigue":
                            # I = "Given the user's context and sensor data, predict the fatigue score (ranges from 0 to 5)"
                            I = "You are a personalized healthcare agent trained to predict fatigue score which ranges from 0 to 5 based on physiological data and user information."
                            A = "The predicted {} level is {}.".format(SUBTASK, f)
                            # fatigue_records.append({'index':index,'Steps':[x[-1] for x in exercise_hist], 'Burned Calorories':[x[-2] for x in exercise_hist], 'Resting Heart Rate(beats/min)':[x[-1] for x in hr_hist], 'SleepMinutes':[x[-1] for x in sleep_hist], 'Mood':m, 'Fatigue':f})

                        subtask_data.append({'index':index,'task':SUBTASK, 'instruction':I, 'input':Q, 'output':A})
                        index += 1
            final_data_list.append(subtask_data)
            
            #save task records to CSV
            # readiness_pmdata = pd.DataFrame(readiness_records)
            # readiness_pmdata.to_csv("pmdata_readiness.csv")
            # stress_pmdata = pd.DataFrame(stress_records)
            # stress_pmdata.to_csv("pmdata_stress.csv")
            # sleep_quality_pmdata = pd.DataFrame(sleep_quality_records)
            # sleep_quality_pmdata.to_csv("pmdata_sleep_quality.csv")
            # fatigue_pmdata = pd.DataFrame(fatigue_records)
            # fatigue_pmdata.to_csv("pmdata_fatigue.csv")
            
    elif DATA == "GLOBEM":
        # SUBTASK = "anxiety" #"depression", "anxiety"
        subtasks = ["anxiety", "depression"]
        # print("[INFO] Dataset:", DATA)
        # print("[INFO] Subtask:", SUBTASK)
        # TODO: add INS-W_2,3,4
        final_data_list = []
        for SUBTASK in subtasks:
            final_data = []
            for insw_idx in tqdm([1,2,3,4]):
                data1 = csv_reader(data_local_path_header + 'data/globem/1.1/INS-W_{}/SurveyData/dep_weekly.csv'.format(insw_idx))
                dict1 = {}
                keys1 = []

                for idx,_data in enumerate(data1):
                    # ['', 'pid', 'date', 'feel_anxious', 'feel_depressed', 'BDI2', 'dep', 'dep_weekly_subscale', 'anx_weekly_subscale', 'dep_weeklysubscale_endterm_merged']
                    if idx == 0:
                        for _idx,_d in enumerate(_data):
                            if _idx == 0:
                                continue
                            keys1.append(_d)
                            dict1[_d] = []
                    else:
                        for _idx, _d in enumerate(_data):
                            dict1[keys1[_idx-1]].append(_d)
                
                data2 = csv_reader(data_local_path_header + 'data/globem/1.1/INS-W_{}/FeatureData/steps.csv'.format(insw_idx))
                dict2 = {}
                keys2 = []

                for idx,_data in enumerate(data2):
                    if idx == 0:
                        for _idx,_d in enumerate(_data):
                            if _idx == 0:
                                continue
                            keys2.append(_d)
                            dict2[_d] = []
                    else:
                        for _idx, _d in enumerate(_data):
                            dict2[keys2[_idx-1]].append(_d)
                
                data3 = csv_reader(data_local_path_header + 'data/globem/1.1/INS-W_{}/FeatureData/sleep.csv'.format(insw_idx))
                dict3 = {}
                keys3 = []

                for idx,_data in enumerate(data3):
                    if idx == 0:
                        for _idx,_d in enumerate(_data):
                            if _idx == 0:
                                continue
                            keys3.append(_d)
                            dict3[_d] = []
                    else:
                        for _idx, _d in enumerate(_data):
                            dict3[keys3[_idx-1]].append(_d)
                
                for idx,pid in enumerate(dict1['pid']):
                    skip = False
                    date = dict1['date'][idx]
                    try:
                        dep = float(dict1['feel_depressed'][idx])
                        anx = float(dict1['feel_anxious'][idx])
                        # print("[{}]".format(idx))
                        # print("[dict1] pid: {}, date: {}, dep: {:.2f}".format(pid, date, dep))
                    except:
                        continue
                    
                    i = 0
                    while True:
                        if dict2['pid'][i] == pid and dict2['date'][i] == date:
                            interests = [
                                "f_steps:fitbit_steps_summary_rapids_maxsumsteps:14dhist", # The maximum daily step count during a time segment.
                                "f_steps:fitbit_steps_summary_rapids_minsumsteps:14dhist", 
                                "f_steps:fitbit_steps_summary_rapids_avgsumsteps:14dhist", 
                                "f_steps:fitbit_steps_summary_rapids_mediansumsteps:14dhist", 
                                "f_steps:fitbit_steps_summary_rapids_stdsumsteps:14dhist" 
                            ]
                            try:
                                maxsumsteps = float(dict2[interests[0]][i])
                                minsumsteps = float(dict2[interests[1]][i])
                                avgsumsteps = float(dict2[interests[2]][i])
                                mediansumsteps = float(dict2[interests[3]][i])
                                stdsumsteps = float(dict2[interests[4]][i])

                                # print("[dict2] pid: {}, date: {}, max: {:.2f}, min: {:.2f}, avg: {:.2f}, median: {:.2f}, std: {:.2f}".format(dict2['pid'][i], dict2['date'][i], maxsumsteps, minsumsteps, avgsumsteps, mediansumsteps, stdsumsteps))
                            except:
                                skip = True
                                break 
                            
                            break
                        else:
                            i += 1
                    
                    i = 0
                    while True:
                        if dict3['pid'][i] == pid and dict3['date'][i] == date:
                            interests = [
                                "f_slp:fitbit_sleep_summary_rapids_avgefficiencymain:14dhist", # Average sleep efficiency for a certain sleep type during a time segment.
                                "f_slp:fitbit_sleep_summary_rapids_avgdurationafterwakeupmain:14dhist", # Average duration the user stayed in bed after waking up for a certain sleep type during a time segment.
                                "f_slp:fitbit_sleep_summary_rapids_avgdurationasleepmain:14dhist", # Average duration the user spent to sleep for a certain sleep type during a time segment.
                                "f_slp:fitbit_sleep_summary_rapids_avgdurationawakemain:14dhist", # Average duration the user stayed awake but still in bed for a certain sleep type during a time segment.
                                "f_slp:fitbit_sleep_summary_rapids_avgdurationtofallasleepmain:14dhist", # Average duration the user spent to fall asleep for a certain sleep type during a time segment.
                                "f_slp:fitbit_sleep_summary_rapids_avgdurationinbedmain:14dhist" 
                            ]
                            try:
                                durafwake = float(dict3[interests[1]][i])
                                dursleep = float(dict3[interests[2]][i])
                                durawake = float(dict3[interests[3]][i])
                                durfall = float(dict3[interests[4]][i])
                                durbed = float(dict3[interests[5]][i])
                                eff = float(dict3[interests[0]][i])

                                # print("[dict3] pid: {}, date: {}, eff: {:.2f}, durafwake: {:.2f}, dursleep: {:.2f}, durawake: {:.2f}, durfall: {:.2f}, durbed: {:.2f}".format(dict3['pid'][i], dict3['date'][i], eff, durafwake, dursleep, durawake, durfall, durbed))
                            except:
                                skip = True
                                break

                            break
                        else:
                            i += 1
                    
                    if skip == True:
                        continue

                    # NEW
                    I = "You are a personalized healthcare agent trained to predict PHQ-4 {} which ranges from 0 to 4 based on physiological data and user information.".format(SUBTASK)

                    if SUBTASK == "depression":
                        # I = "Given the user's context and sensor data, predict the PHQ-4 depression level (from 0 to 4)"
                        A = "The predicted PHQ-4 {} score is {}.".format(SUBTASK, int(dep))
                    elif SUBTASK == "anxiety":
                        # I = "Given the user's context and sensor data, predict the PHQ-4 anxiety level (from 0 to 4)"
                        A = "The predicted PHQ-4 {} score is {}.".format(SUBTASK, int(anx))
                    
                    Q = "The recent 14-days sensor readings show: [Steps] is {}. [Sleep] efficiency, duration the user stayed in bed after waking up, duration the user spent to sleep, duration the user stayed awake but still in bed, duration the user spent to fall asleep are {}, {}, {}, {}, {} mins in average; What would be the PHQ-4 {} score?".format(avgsumsteps, eff, durafwake, dursleep, durawake, durfall, SUBTASK)

                    final_data.append({'task': SUBTASK,'instruction':I, 'input':Q, 'output':A})  
            final_data_list.append(final_data)
                                        
    elif DATA == "AW_FB":
        DATA_PATH = data_local_path_header + "data/harvard_dataverse"
        # SUBTASK = "activity"
        # print("[INFO] Generating datasets for AW_FB (Harvard Dataverse) ...")
        # print("[INFO] Subtask:", SUBTASK)
        final_data_list = []
        with open('{}/aw_fb_data.csv'.format(DATA_PATH), newline='') as csvfile:
            # csv_reader = csv.reader(csvfile, delimiter=',', quotechar='"')
            df = pd.read_csv('{}/aw_fb_data.csv'.format(DATA_PATH), quotechar='"')
            # final_data = []
            activities = []
            subtask1 = []
            subtask2 = []
            # for idx,row in enumerate(csv_reader):
            for idx, row in df.iterrows(): 
                if idx == 0:
                    continue

                # try:
                # tmp = row[0].split(",")
                # print(tmp)
                age = int(row.iloc[2])
                gender = int(row.iloc[3])
                if gender == 1:
                    gender = "M"
                elif gender == 0:
                    gender = "F"
                height = str(row.iloc[4]) + " cm"
                weight = str(row.iloc[5]) + " kg"
                steps = float(row.iloc[6])
                heart_rate = float(row.iloc[7])
                calories = float(row.iloc[8])
                distance = float(row.iloc[9])
                entropy_heart = float(row.iloc[10])
                entropy_steps = float(row.iloc[11])
                resting_heart = float(row.iloc[12])
                corr_heart_steps = float(row.iloc[13])
                norm_heart = float(row.iloc[14])
                intensity_karvonen = float(row.iloc[15]) 
                sd_norm_heart = float(row.iloc[16])
                steps_times_distance = float(row.iloc[17]) 
                device = row.iloc[18]
                
                # if len(row) == 3:
                #     activity = row[0].split(",")[-1] + " " + row[1] + " " + row[2]
                # else:
                activity = row.iloc[19] # label

                activity = activity.replace('"', "")

                if activity == "Self Pace walk":
                    met_value = 2
                elif activity == "Sitting":
                    met_value = 1
                elif activity == "Lying":
                    met_value = 0.5
                elif activity == "Running 7 METs":
                    met_value = 7
                elif activity == "Running 5 METs":
                    met_value = 5
                elif activity == "Running 3 METs":
                    met_value = 3

                duration = calories / (met_value * float(row.iloc[5]) / 200)

                activities.append(activity)

                # I = "Answer this question truthfully"
        
                # if SUBTASK == "activity":
                    # Q = "Predict the {} type among ['Self Pace Walk', 'Sitting', 'Lying', 'Running 7 METs', 'Sitting', 'Running 5 METs', 'Running 3 METs'] given the following information. [Age]: {}, [Gender]: {}, [Height]: {}, [Weight]: {}, [Steps]: {:.2f} steps, [Burned Calorories]: {:.2f} calories, [Heart Rate]: {:.2f} beats/min".format(SUBTASK, age, gender, height, weight, steps, calories, heart_rate)
                    # A = "{}".format(activity.replace('\"', ''))
                
                    # NEW
                I = "You are a personalized healthcare agent trained to predict the type of activity among ['Self Pace Walk', 'Sitting', 'Lying', 'Running 7 METs', 'Running 5 METs', 'Running 3 METs'] based on physiological data and user information."
                Q = "The recent sensor readings show: [Steps]: {:.2f} steps, [Burned Calorories]: {:.2f} calories, [Heart Rate]: {:.2f} beats/min; What would be the predicted activity type?".format(steps, calories, heart_rate)
                A = "The predicted activity type is {}".format(activity.replace('\"', ''))
                subtask1.append({'task':'activity','instruction':I, 'input':Q, 'output':A})
                # elif SUBTASK == "calories":
                    # Q = "Predict the burned {} given the following information. [Age]: {}, [Gender]: {}, [Height]: {}, [Weight]: {}, [Steps]: {:.2f} steps, [Heart Rate]: {:.2f} beats/min".format(SUBTASK, age, gender, height, weight, steps, heart_rate)
                    # A = "{}".format(calories)

                    # NEW
                I = "You are a personalized healthcare agent trained to predict the calorie burn based on physiological data and user information."
                Q = "The recent sensor readings show: [Steps]: {:.2f} steps, [Heart Rate]: {:.2f} beats/min, [Duration]: {:.2f}, [Activity Type]: {}, [MET Value]: {}; What would be the predicted calorie burn?".format(steps, heart_rate, duration, activity, met_value)
                A = "The predicted calorie burn is {}.".format(calories)
                subtask2.append({'task': 'calories','instruction':I, 'input':Q, 'output':A})
            final_data_list.append(subtask1)
            final_data_list.append(subtask2)
                
                # except Exception as e:
                #     print(e)
                #     continue

In [ ]:
final_data_list[1][0:5]

##  Train and evaluation data split

In [ ]:
fatigue_train = []
readiness_train = []
stress_train = []
sleep_quality_train = []

for n, SUBTASK in enumerate(['sleep_quality', 'stress', 'readiness', 'fatigue']): 
 
# for n, SUBTASK in enumerate(["anxiety", "depression"]): 
# for n, SUBTASK in enumerate(['activity', 'calories']): 
    final_data = final_data_list[n]
    
    N = len(final_data)
    final_train_data = []
    final_eval_data = []

    num_sd = {0:0, 1:0}
    num_sd_eval = {0:0, 1:0}


    import random
    random.seed(123)
    random.shuffle(final_data)
    eval_idx = 1
    for n,fd in enumerate(final_data):
        if n < int(N*0.5):
            if SUBTASK == "sleep_disorder":
                x = int(fd['output'].split()[-1].replace(".", ""))
                if x == 0 and num_sd[x] < 13:
                    fd['question'] = fd['input']
                    fd['answer'] = fd['output']
                    del fd['input']
                    del fd['output']
                    final_train_data.append(fd)
                    num_sd[x] += 1
                elif x == 1 and num_sd[x] < 13:
                    fd['question'] = fd['input']
                    fd['answer'] = fd['output']
                    del fd['input']
                    del fd['output']
                    final_train_data.append(fd)
                    num_sd[x] += 1
            
            elif "ibis" in SUBTASK:
                print(fd)
                fd['question'] = fd['input']
                fd['answer'] = fd['output']
                x = fd['output']
                del fd['input']
                del fd['output']
                final_train_data.append(fd)


            else:
                # if SUBTASK == "fatigue":
                # for i in range(len(globals()[f"{SUBTASK}_records"])):
                #     if globals()[f"{SUBTASK}_records"][i]["index"] == fd["index"]:
                #         globals()[f"{SUBTASK}_train"].append(globals()[f"{SUBTASK}_records"][i])
                #         break

                # elif SUBTASK == "readiness":
                #     readiness_train.append(readiness_records[fd["index"]])
                # elif SUBTASK == "stress":
                #     stress_train.append(stress_records[fd["index"]])
                # elif SUBTASK == "sleep_quality":
                #     sleep_quality_train.append(sleep_quality_records[fd["index"]])
                    
                fd['question'] = fd['input']
                fd['answer'] = fd['output']
                del fd['input']
                del fd['output']
                final_train_data.append(fd)
        else:
            if SUBTASK == "sleep_disorder":
                
                if int(fd['output'].split()[-1].replace(".", "")) == 0 and num_sd_eval[int(fd['output'].split()[-1].replace(".", ""))] < 25:
                    x = int(fd['output'].split()[-1].replace(".", ""))
                    fd['no'] = eval_idx
                    fd['question'] = fd['input']
                    fd['answer'] = fd['output']
                    del fd['input']
                    del fd['output']

                    final_eval_data.append(fd)
                    eval_idx += 1
                    num_sd_eval[x] += 1
                
                elif int(fd['output'].split()[-1].replace(".", "")) == 1 and num_sd_eval[int(fd['output'].split()[-1].replace(".", ""))] < 25:
                    x = int(fd['output'].split()[-1].replace(".", ""))
                    fd['no'] = eval_idx
                    fd['question'] = fd['input']
                    fd['answer'] = fd['output']
                    del fd['input']
                    del fd['output']

                    final_eval_data.append(fd)
                    eval_idx += 1
                    num_sd_eval[x] += 1
                
            else:
                if eval_idx < 300:
                    fd['no'] = eval_idx
                    fd['question'] = fd['input']
                    fd['answer'] = fd['output']
                    del fd['input']
                    del fd['output']

                    final_eval_data.append(fd)
                    eval_idx += 1
            
    # 3-shot
    json_object_train_3 = json.dumps(final_train_data[:3], indent=4)
    # 5-shot
    json_object_train_5 = json.dumps(final_train_data[:5], indent=4)
    # 10-shot
    json_object_train_10 = json.dumps(final_train_data[:10], indent=4)
    # 25-shot
    json_object_train_25 = json.dumps(final_train_data[:25], indent=4)

    # json_object_eval = json.dumps(final_eval_data, indent=4)
    # fine-tune
    json_object_train_all = json.dumps(final_train_data, indent=4)

    # # zero-shot
    # json_object_train_all = json.dumps(final_train_data, indent=4)
    json_object_eval = json.dumps(final_eval_data, indent=4)
    # write
    
    # train
    # with open("zero-shot-predict/{}_{}_predict_3.json".format(DATA, SUBTASK), "w") as outfile_train_3:
    #     outfile_train_3.write(json_object_train_all)
    os.makedirs('train', exist_ok=True)
    with open("train/{}_{}_train_3.json".format(DATA.lower(), SUBTASK), "w") as outfile_train_3:
        outfile_train_3.write(json_object_train_3)

    with open("train/{}_{}_train_5.json".format(DATA.lower(), SUBTASK), "w") as outfile_train_5:
        outfile_train_5.write(json_object_train_5)

    with open("train/{}_{}_train_10.json".format(DATA.lower(), SUBTASK), "w") as outfile_train_10:
        outfile_train_10.write(json_object_train_10)

    with open("train/{}_{}_train_25.json".format(DATA.lower(), SUBTASK), "w") as outfile_train_25:
        outfile_train_25.write(json_object_train_25)

    with open("train/{}_{}_train_all.json".format(DATA.lower(), SUBTASK), "w") as outfile_train_all:
        outfile_train_all.write(json_object_train_all)


    # #save task_train to CSV
    # readiness_pmdata = pd.DataFrame(readiness_train)
    # readiness_pmdata.to_csv("Datasets/PMdata/realdata/pmdata_readiness.csv")
    # stress_pmdata = pd.DataFrame(stress_train)
    # stress_pmdata.to_csv("Datasets/PMdata/realdata/pmdata_stress.csv")
    # sleep_quality_pmdata = pd.DataFrame(sleep_quality_train)
    # sleep_quality_pmdata.to_csv("Datasets/PMdata/realdata/pmdata_sleep_quality.csv")
    # fatigue_pmdata = pd.DataFrame(fatigue_train)
    # fatigue_pmdata.to_csv("Datasets/PMdata/realdata/pmdata_fatigue.csv")

    # eval
    os.makedirs('eval/{}_{}'.format(DATA.lower(), SUBTASK), exist_ok=True)
    with open("eval/{}_{}/step1.json".format(DATA.lower(), SUBTASK), "w") as outfile_eval:
        outfile_eval.write(json_object_eval)


    # n_samples = 100

    # if MODE == "eval":
    #     elif DATA == "":
    #         pass


## AW_FB Activity Mapping

In [ ]:
import json
def json_reader(file_name):
    f = open(file_name)
    data = json.load(f)
    f.close()
    return data

activity_mapping = {
    "Self Pace walk": 0,
    "Sitting": 1,
    "Lying": 2,
    "Running 7 METs": 3,
    "Running 5 METs": 4,
    "Running 3 METs": 5
}
def updated_activity(data):
    updated_data = []
    for instance in data:
        # print(instance)
        task = instance['task']
        instruction = "You are a personalized healthcare agent trained to predict the type of activity among {0:'Self Pace Walk', 1:'Sitting', 2:'Lying', 3:'Running 7 METs', 4:'Running 5 METs', 5:'Running 3 METs'} based on physiological data and user information."
        question = instance['question']
        answer = instance["answer"].replace("The predicted activity type is ", "").strip()
        answer_index = activity_mapping[answer]
        
        updated_data.append({
            "task": task,
            "instruction": instruction,
            "question": question,
            "answer": f"The predicted activity type is {answer_index}."
        })
    return updated_data

few_shots = [3, 5, 10, 25, "all"]
for num in few_shots:
    AW_activity = json_reader("train/aw_fb_activity_train_{}.json".format(str(num)))
    step1 = json_reader("eval/aw_fb_activity/step1.json")
    # print(len(AW_activity))
    
    # updating train
    updated_data = updated_activity(AW_activity)
    output_file = f"train_task_disambiguation/aw_fb_activity_train_{num}_updated.json"
    with open(output_file, 'w') as f: json.dump(updated_data, f, indent=4)
    
    # updating eval
    updated_data = updated_activity(step1)
    output_file = "eval_task_disambiguation/aw_fb_activity_updated/step1.json"
    os.makedirs('eval_task_disambiguation/aw_fb_activity_updated', exist_ok=True)
    with open(output_file, 'w') as f: json.dump(updated_data, f, indent=4)
        

# Ensure class-balance in Few-shot examples for task (sleep disorder)

In [ ]:
# Cell removed